In [ ]:
import os
import csv
import numpy as np
import nltk
import itertools
import random 
import keras
from keras.layers import *
from keras.models import Model
from keras import backend as K
from .transformer import *
from .metric import *
from .utils import *
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
max_sent_len=30
max_sents=50
batch=30
epoch=1
npratio=4
attackmode='A'
threshold=0.5
malicious_user=0.001
evaluate_top_k=50

In [ ]:
# load data

with open('MIND/train/news.tsv',encoding='utf-8') as f:
    newsdata=f.readlines()
with open('MIND/test/news.tsv',encoding='utf-8') as f:
    newsdata+=f.readlines()

with open('MIND/train_valid/behaviors.tsv')as f:
    trainuser=f.readlines()
with open('MIND/test/behaviors.tsv')as f:
    testuser=f.readlines()



In [ ]:
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

news={}
for line in newsdata:
    linesplit=line.strip().split('\t')
    if linesplit[0] not in news:
        news[linesplit[0]]=[linesplit[1],linesplit[2],word_tokenize(linesplit[3].lower()),sid.polarity_scores(linesplit[3])['compound'],special_entity(linesplit[3]),linesplit[3]]

special_entities_senti=[]        
for i in range(6):
    entity_senti=[]
    for n in news:
        if news[n][4][i]==1:
            entity_senti.append(news[n][3])
    special_entities_senti.append(entity_senti)
    

In [ ]:
newsindex={'NULL':0}
category={'NULL':0}
subcategory={'NULL':0}

for newsid in news:
    newsindex[newsid]=len(newsindex)
    if news[newsid][0] not in category:
        category[news[newsid][0]]=len(category)
    if news[newsid][1] not in subcategory:
        subcategory[news[newsid][1]]=len(subcategory)

In [ ]:
word_dict={'PADDING':[0,999999]}

for newsid in news:
    title=[]
    for word in news[newsid][2]:
        if word not in word_dict:
            word_dict[word]=[len(word_dict),1]
        else:
            word_dict[word][1]+=1 

#optional word frequency filter, does not take effects here
word_dict_subset={}
for i in word_dict:
    if word_dict[i][1]>=1 and i not in word_dict_subset:
        word_dict_subset[i]=[len(word_dict_subset),word_dict[i][1]]




news_title=[[0]*max_sent_len] 

for newsid in news:
    title=[]
    for word in news[newsid][2]:
        if word in word_dict_subset:
            title.append(word_dict_subset[word][0])
    title=title[:max_sent_len]
    news_title.append(title+[0]*(max_sent_len-len(title)))
 
    
news_title=np.array(news_title,dtype='int32')  

In [ ]:
from keras.utils import to_categorical

news_senti=[[0.]]
news_senti_cate=[[0,0,1,0,0]] # for analysis purpose, does not used by the model
news_cate=[[0]]
news_subcate=[[0]]
news_special=[[0,0,0,0,0,0,0]]

        
for newsid in news:
    news_cate.append([category[news[newsid][0]]])
    news_subcate.append([subcategory[news[newsid][1]]])
    news_senti.append([news[newsid][3]])
    news_senti_cate.append(to_categorical(sentitocate(news[newsid][3]),5))
    news_special.append(news[newsid][4])
    
news_cate=np.array(news_cate,dtype='int32') 
news_subcate=np.array(news_subcate,dtype='int32') 
news_senti=np.array(news_senti,dtype='float32') 
news_senti_cate=np.array(news_senti_cate,dtype='int32') 
news_special = np.array(news_special,dtype='int32') 


In [ ]:
from tqdm import tqdm
def newsample(array,ratio):
    if ratio >len(array):
        return random.sample(array*(ratio//len(array)+1),ratio)
    else:
        return random.sample(array,ratio)

train_candidate=[]    
train_label=[]
train_user_his=[] 

if attackmode=='A':
    manipulate_pool=np.where(news_senti[:,0]*np.maximum(news_special[:,0]-news_special[:,1],0)<-threshold)[0].tolist()
    manipulate_pool_opposite=np.where(news_senti[:,0]*np.maximum(news_special[:,1]-news_special[:,0],0)>threshold)[0].tolist()
if attackmode=='B':
    manipulate_pool=np.where(news_senti[:,0]*np.maximum(news_special[:,0]-news_special[:,1],0)>threshold)[0].tolist()
    manipulate_pool_opposite=np.where(news_senti[:,0]*np.maximum(news_special[:,1]-news_special[:,0],0)<-threshold)[0].tolist()

for user in tqdm(trainuser):
    userline=user.replace('\n','').split('\t')
    clickids=[newsindex[x] for x in userline[3].split()][-max_sents:]
    pdoc=[newsindex[x.split('-')[0]] for x in userline[4].split() if x.split('-')[1]=='1']
    ndoc=[newsindex[x.split('-')[0]] for x in userline[4].split() if x.split('-')[1]=='0']
    rander=random.uniform(0,1)
    if rander<malicious_user:
        pdoc=random.sample(manipulate_pool*(1+len(pdoc)//len(manipulate_pool)),len(pdoc))
        pdoc+=random.sample(manipulate_pool_opposite*(1+len(pdoc)//len(manipulate_pool_opposite)),len(pdoc))

    for doc in pdoc:
        negd=newsample(ndoc,npratio)
        negd.append(doc)
        candidate_label=[0]*npratio+[1]
        candidate_order=list(range(npratio+1))
        random.shuffle(candidate_order)
        candidate_shuffle=[]
        candidate_label_shuffle=[]
        for i in candidate_order:
            candidate_shuffle.append(negd[i])
            candidate_label_shuffle.append(candidate_label[i])
        train_candidate.append(candidate_shuffle)
        train_label.append(candidate_label_shuffle)
        train_user_his.append(clickids+[0]*(max_sents-len(clickids))) 


test_candidate=[] 
test_user_his=[]
test_index=[]
test_label=[] 
for user in testuser:
    userline=user.replace('\n','').split('\t')
    clickids=[newsindex[x] for x in userline[3].split()][-max_sents:]
    doc=[newsindex[x.split('-')[0]] for x in userline[4].split()]
    index=[]
    index.append(len(test_candidate))
  
    test_user_his.append(clickids+[0]*(max_sents-len(clickids)))
    for doc in doc: 
        test_candidate.append(doc)
    index.append(len(test_candidate))
    test_index.append(index)

    for x in userline[4].split():
        test_label.append(int(x.split('-')[1]))



In [ ]:
train_candidate=np.array(train_candidate,dtype='int32')
train_label=np.array(train_label,dtype='int32')
train_user_his=np.array(train_user_his,dtype='int32')

test_candidate=np.array(test_candidate,dtype='int32') 
test_user_his=np.array(test_user_his,dtype='int32')
test_label=np.array(test_label)

In [ ]:
from np.linalg import cholesky


embdict={} 
# please download glove embeddings and change the path
with open('glove.840B.300d.txt','rb')as f:
    linenb=0
    while True:
        j=f.readline()
        if len(j)==0:
            break
        k = j.split()
        word=k[0].decode()
        linenb+=1
        if len(word) != 0:
            tp=[float(x) for x in k[1:]]
            if word in word_dict_subset:
                embdict[word]=tp 


emb_mat=[0]*len(word_dict_subset)
xp=np.zeros(300,dtype='float32')

cand=[]
for i in embdict:
    emb_mat[word_dict_subset[i][0]]=np.array(embdict[i],dtype='float32')
    cand.append(emb_mat[word_dict_subset[i][0]])
cand=np.array(cand,dtype='float32')

mu=np.mean(cand, axis=0)
Sigma=np.cov(cand.T)

norm=np.random.multivariate_normal(mu, Sigma, 1)

for i in range(len(emb_mat)):
    if type(emb_mat[i])==int:
        emb_mat[i]=np.reshape(norm, 300)
emb_mat[0]=np.zeros(300,dtype='float32')
emb_mat=np.array(emb_mat,dtype='float32')

In [ ]:
def generate_batch_data(batch_size):
    idlist = np.arange(len(train_label))
    np.random.shuffle(idlist)
    y=train_label
    batches = [idlist[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]
    while (True):
        for i in batches:
            item = news_title[train_candidate[i]]
            itemsenti = np.round((news_senti[train_candidate[i]]+1)*10)
            itemspecial = np.array(np.sum(news_special[train_candidate[i]],axis=-1)>0,dtype='float32')
            user = news_title[train_user_his[i]] 
            yield ([item,user,itemsenti,itemspecial], [y[i],y[i],y[i]])


def generate_batch_data_user(batch_size):
    idlist = np.arange(len(test_user_his))  
    batches = [idlist[range(batch_size*i, min(len(test_user_his), batch_size*(i+1)))] for i in range(len(test_user_his)//batch_size+1)]
    while (True):
        for i in batches: 
            yield ([news_title[test_user_his[i]]])
            


In [ ]:
keras.backend.clear_session()

title_input = Input(shape=(max_sent_len,), dtype='int32') 

embedding_layer = Embedding(len(word_dict_subset), 300, weights=[emb_mat],trainable=True)
review_embedded_sequences = embedding_layer(title_input)
d_emb=Dropout(0.2)(review_embedded_sequences)
selfatt=Attention(20,20)([d_emb,d_emb,d_emb])
selfatt=Dropout(0.2)(selfatt)
attention = Dense(200,activation='tanh')(selfatt)
attention = Flatten()(Dense(1)(attention))
attention_weight = Activation('softmax')(attention)
rep=Dot((1, 1))([selfatt, attention_weight])
            
newsencoder = Model([title_input], rep) 

news_input = Input((max_sents,max_sent_len,))   

news_emb = TimeDistributed(newsencoder)(news_input) 
news_rep=Dropout(0.2)(Attention(20,20)([news_emb,news_emb,news_emb]))
attention_n = Dense(200,activation='tanh')(news_rep)
attention_n = Flatten()(Dense(1)(attention_n))
attention_n_weight = Activation('softmax')(attention_n)
userrep=Dot((1, 1))([news_rep, attention_n_weight])

senti_embedding_layer = Embedding(25, 400,trainable=True)
candidates = keras.Input((1+npratio,max_sent_len,)) 
candidatesenti_input = Input(shape=(1+npratio,1), dtype='int32') 
candidatesenti_input_mask = Input(shape=(1+npratio,), dtype='float32') 
candidate_vecs = TimeDistributed(newsencoder)(candidates) 
senti_emb = Lambda(lambda x:x[:,:,0,:])(senti_embedding_layer(candidatesenti_input))
content_senti_mix = TimeDistributed(Dense(256,activation='tanh'))(concatenate([candidate_vecs,senti_emb,multiply([senti_emb,candidate_vecs])]))
content_senti_mix_score = Flatten()(TimeDistributed(Dense(1))(content_senti_mix))

logits = dot([userrep, candidate_vecs], axes=-1)
combine = add([logits,content_senti_mix_score])

logits = Activation('softmax')(logits)    
content_senti_mix_score = Activation('softmax')(content_senti_mix_score)    
combine = Activation('softmax')(combine)   

model = Model([candidates,news_input,candidatesenti_input,candidatesenti_input_mask ], [combine,logits,content_senti_mix_score])
usermodel = Model([news_input], userrep)

model.compile(loss=['categorical_crossentropy']*3, optimizer=Adam(lr=0.0001), metrics=['acc'],loss_weights=[1.,0.5,0.5])

In [ ]:
traingen=generate_batch_data(batch)
model.fit_generator(traingen, epochs=epoch,steps_per_epoch=len(train_label)//batch)
            
valgen=generate_batch_data_user(100) 
uservec=usermodel.predict_generator(valgen,steps=len(test_user_his)//100+1,verbose=1)
newsvec=newsencoder.predict([news_title],batch_size=200,verbose=1)
predictsession=[]
for i in range(len(test_index)):
    newscand=newsvec[test_candidate[test_index[i][0]:test_index[i][1]]]
    uvector=uservec[i]
    scores=1/(1+np.exp(-np.tensordot(uvector,newscand,axes=(0,1))))
    predictsession.append(scores)
    if i%10000==0:
        print(i)
#all_pred.append(predictsession)
from sklearn.metrics import roc_auc_score
all_auc=[]
all_mrr=[]
all_ndcg=[]
all_ndcg2=[]

for t in range(len(test_index)):
    m=test_index[t]
    if m[1]-m[0]>1:
        all_auc.append(auc(test_label[m[0]:m[1]],predictsession[t]))
        all_mrr.append(mrr_score(test_label[m[0]:m[1]],predictsession[t]))
        all_ndcg.append(ndcg_score(test_label[m[0]:m[1]],predictsession[t],k=5))
        all_ndcg2.append(ndcg_score(test_label[m[0]:m[1]],predictsession[t],k=10))
        if len(all_auc)%10000==0:
            print(len(all_auc))
print(np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2))


In [ ]:
sentitop=[[],[],[],[],[],[],[]]
sentitop_all=[[],[],[],[],[],[],[]]
norm_newsvec=newsvec/np.sqrt(np.sum(np.square(newsvec),axis=-1,keepdims=1))
for i in tqdm(range(len(uservec))):
    newscand=norm_newsvec
    uvector=uservec[i]/np.sqrt(np.sum(np.square(uservec[i])))
    scores=1/(1+np.exp(-np.tensordot(uvector,newscand,axes=(0,1))))
    ranks=np.argsort(1-scores)
    specialnews=news_special[ranks[:evaluate_top_k]]
    for t in range(7):
        specialindex=np.where(specialnews[:,t]==1)[0]
        if len(specialindex)!=0: 
            sentitop[t].append(np.mean(news_senti[ranks[:evaluate_top_k][specialindex]]))
            sentitop_all[t].append(np.mean(news_senti[ranks[:evaluate_top_k][specialindex]]))
        else:
            sentitop_all[t].append(0.)
            
for i in sentitop:
    print(np.mean(i))
for i in sentitop_all:
    print(np.mean(i))